In [1]:
import pandas as pd

In [9]:
df = pd.read_csv("Dataset_09.csv")

In [10]:
df_q2 = df.copy()

In [11]:
df_q2 = df_q2.dropna() # 특정 row 에 결측치가 하나라도 있는 경우 제거

In [12]:
df_q2.isna().sum().sum()

0

In [13]:
df_q2["Age"].describe()

count    1995.000000
mean       38.164912
std        18.540405
min         7.000000
25%        22.000000
50%        38.000000
75%        54.000000
max        70.000000
Name: Age, dtype: float64

In [14]:
df_q2["Age_gr"] = pd.cut(df_q2["Age"], 
                              bins = [0,20, 30, 40, 50, 60, 80],
                              labels = [10, 20, 30, 40, 50 ,60])

In [15]:
pd.crosstab(df_q2["Age"], df_q2["Age_gr"])

Age_gr,10,20,30,40,50,60
Age,,,,,,
7.0,28,0,0,0,0,0
8.0,30,0,0,0,0,0
9.0,33,0,0,0,0,0
10.0,31,0,0,0,0,0
11.0,40,0,0,0,0,0
...,...,...,...,...,...,...
66.0,0,0,0,0,0,28
67.0,0,0,0,0,0,32
68.0,0,0,0,0,0,27


In [16]:
from scipy.stats import chi2_contingency
import pandas as pd

In [22]:
result1 = chi2_contingency(pd.crosstab(df_q2["satisfaction"],df_q2["Age_gr"]))
result2 = chi2_contingency(pd.crosstab(df_q2["satisfaction"],df_q2["Gender"]))
result3 = chi2_contingency(pd.crosstab(df_q2["satisfaction"],df_q2["Customer_Type"]))
result4 = chi2_contingency(pd.crosstab(df_q2["satisfaction"],df_q2["Class"]))

In [23]:
print(result1[0], round(result1[1], 3))
print(result2[0], round(result2[1], 3))
print(result3[0], round(result3[1], 3))
print(result4[0], round(result4[1], 3))


2.287008134344471 0.808
1066.7824258873009 0.0
0.46919210489755214 0.493
0.3115492553250723 0.856


In [24]:
int(result2[0])

1066

//------------------------------------------------------------------

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [41]:
df_train, df_test = train_test_split(df.dropna()
                                     , train_size = 0.7, random_state = 123)
print(len(df_train), len(df_test))
df_train.head(2)

1396 599


,satisfaction,Gender,Age,Customer_Type,Class,Flight_Distance,Seat_comfort,Food_and_drink,Inflight_wifi_service,Inflight_entertainment,Onboard_service,Leg_room_service,Baggage_handling,Cleanliness,Departure_Delay_in_Minutes,Arrival_Delay_in_Minutes
173,dissatisfied,Male,60.0,Loyal Customer,Eco,2548.0,3.0,3.0,1.0,3.0,3.0,2.0,3.0,5.0,0.0,0.0
1002,dissatisfied,Female,35.0,Loyal Customer,Eco,1886.0,4.0,4.0,4.0,4.0,3.0,5.0,5.0,5.0,0.0,18.0


In [42]:
X = df_train.loc[:, "Flight_Distance":] # Flight_Distance 부터 끝까지 
X.head(2)

,Flight_Distance,Seat_comfort,Food_and_drink,Inflight_wifi_service,Inflight_entertainment,Onboard_service,Leg_room_service,Baggage_handling,Cleanliness,Departure_Delay_in_Minutes,Arrival_Delay_in_Minutes
173,2548.0,3.0,3.0,1.0,3.0,3.0,2.0,3.0,5.0,0.0,0.0
1002,1886.0,4.0,4.0,4.0,4.0,3.0,5.0,5.0,5.0,0.0,18.0


In [43]:
y = (df_train["satisfaction"] == "dissatisfied") + 0
y.head()

173     1
1002    1
1537    1
1105    1
1230    1
Name: satisfaction, dtype: int32

In [46]:
model = LogisticRegression(solver = "liblinear", random_state = 123).fit(X, y)
model

LogisticRegression(random_state=123, solver='liblinear')

In [49]:
pred = model.predict(df_test.loc[:, "Flight_Distance":])
pred[:5]

array([1, 1, 0, 0, 1])

In [54]:
f1_score(y_true = (df_test["satisfaction"] == "dissatisfied") + 0, 
         y_pred = pred)

0.7771084337349397